In [ ]:
# --- 기본 및 데이터 처리 라이브러리 ---
import gc  # Garbage Collector: 메모리 관리를 위한 라이브러리
import glob  # 파일 경로명을 이용해 원하는 파일들을 찾아주는 라이브러리
import os  # 운영체제와 상호작용하기 위한 라이브러리 (파일 경로 등)
import json  # JSON 형식의 파일을 읽고 쓰기 위한 라이브러리
import pprint  # 데이터 구조를 예쁘게 출력하기 위한 라이브러리

# --- 시각화 라이브러리 ---
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 핵심 라이브러리

# --- 수치 계산 및 데이터 분석 라이브러리 ---
import numpy as np  # 다차원 배열 및 고성능 수치 계산을 위한 라이브러리
import pandas as pd  # 데이터프레임(표 형태의 데이터)을 다루기 위한 핵심 라이브러리

# --- 병렬 처리 및 유틸리티 라이브러리 ---
from joblib import Parallel, delayed  # 작업을 병렬로 처리하여 속도를 높이기 위한 라이브러리
from tqdm import tqdm  # 반복문의 진행 상태를 시각적인 막대로 보여주는 라이브러리
from PIL import Image  # 이미지 파일을 열고 다루기 위한 라이브러리 (Pillow)

# --- Jupyter Notebook 설정 ---
# matplotlib으로 그린 그래프를 노트북 셀 바로 아래에 표시하도록 설정하는 매직 명령어
%matplotlib inline

# --- Pandas 출력 옵션 설정 ---
# 데이터프레임을 출력할 때 보여줄 최대 행과 열의 개수를 설정
pd.options.display.max_rows = 128
pd.options.display.max_columns = 128

In [ ]:
# --- 시각화 그래프의 기본 크기를 지정 ---
plt.rcParams['figure.figsize'] = (12, 9)

In [ ]:
# --- 데이터 디렉터리의 내용을 확인 ---
os.listdir('../input/test/')

- '../input/test/'
  - Kaggle 노트북 환경에서 모든 대회 데이터는 ../input/ 폴더 아래에 위치합니다. 따라서 이 경로는 대회의 test 데이터셋 폴더를 가리킵니다.

In [ ]:
train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')
sample_submission = pd.read_csv('../input/test/sample_submission.csv')

In [ ]:
labels_breed = pd.read_csv('../input/breed_labels.csv')
labels_state = pd.read_csv('../input/state_labels.csv')
labels_color = pd.read_csv('../input/color_labels.csv')

In [ ]:
train_image_files = sorted(glob.glob('../input/train_images/*.jpg'))
train_metadata_files = sorted(glob.glob('../input/train_metadata/*.json'))
train_sentiment_files = sorted(glob.glob('../input/train_sentiment/*.json'))

print('num of train images files: {}'.format(len(train_image_files)))
print('num of train metadata files: {}'.format(len(train_metadata_files)))
print('num of train sentiment files: {}'.format(len(train_sentiment_files)))


test_image_files = sorted(glob.glob('../input/test_images/*.jpg'))
test_metadata_files = sorted(glob.glob('../input/test_metadata/*.json'))
test_sentiment_files = sorted(glob.glob('../input/test_sentiment/*.json'))

print('num of test images files: {}'.format(len(test_image_files)))
print('num of test metadata files: {}'.format(len(test_metadata_files)))
print('num of test sentiment files: {}'.format(len(test_sentiment_files)))

CSV 파일은 그 자체로 **"하나의 완성된 표"**이고, 비정형 데이터는 **"수많은 개별 재료의 목록"**이기 때문에 비정형파일만 파일의 개수를 확인

## 비정형 데이터 파일 (.jpg, .json): "수많은 재료의 목록"을 생성
- 처리 방식: glob.glob()은 수천 개에 달하는 개별 파일의 내용을 읽는 것이 아니라, 파일들의 경로(위치)만 찾아서 리스트(list)로 만듭니다. 아직은 어떤 정보도 처리하지 않은, 말 그대로 "재료 목록"만 만든 상태입니다.

- 확인 대상: 이 단계에서 가장 중요한 정보는 "그래서 파일이 총 몇 개나 있는가?" 입니다. 이 개수를 통해 데이터의 전체 규모를 파악하고, train.csv 파일의 행 개수와 비교하며 데이터가 누락되지 않았는지 1차적으로 점검할 수 있습니다.

- 점검 방법: 따라서 리스트에 들어있는 경로가 총 몇 개인지 len() 함수를 사용해 개수를 세어 출력하는 것이 가장 효과적인 첫 번째 점검 방법입니다.



In [ ]:
# matplotlib 그래프의 시각적 스타일을 'ggplot'으로 설정 (R언어에서 유래한 인기 스타일)
plt.style.use('ggplot')


# --- 1. 이미지(Images) 데이터 커버리지 확인 ---
# 원본 train 데이터에서 'PetID' 컬럼만 추출하여 기준이 될 ID 목록 생성
train_df_ids = train[['PetID']]
# 원본 train 데이터의 전체 반려동물 수를 출력
print(train_df_ids.shape)

# 이전에 수집한 이미지 파일 경로 리스트를 데이터프레임으로 변환
train_df_imgs = pd.DataFrame(train_image_files)
train_df_imgs.columns = ['image_filename'] # 컬럼 이름 지정
# 파일명에서 PetID를 추출. 예: '../.../a1b2c3d4-1.jpg' -> 'a1b2c3d4'
train_imgs_pets = train_df_imgs['image_filename'].apply(lambda x: x.split('/')[-1].split('-')[0])
# 추출한 PetID를 'PetID'라는 새 컬럼으로 추가
train_df_imgs = train_df_imgs.assign(PetID=train_imgs_pets)
# 이미지가 있는 반려동물의 고유한 ID 개수를 출력
print(len(train_imgs_pets.unique()))

# 이미지 파일의 PetID와 원본 train 데이터의 PetID 사이의 교집합(공통 ID)을 구함
pets_with_images = len(np.intersect1d(train_imgs_pets.unique(), train_df_ids['PetID'].unique()))
# 원본 데이터의 모든 반려동물 중, 이미지가 있는 동물의 비율을 계산하여 출력
print('fraction of pets with images: {:.3f}'.format(pets_with_images / train_df_ids.shape[0]))


# --- 2. 메타데이터(Metadata) 커버리지 확인 ---
# (위 이미지 데이터 처리와 과정이 거의 동일)
train_df_ids = train[['PetID']]
train_df_metadata = pd.DataFrame(train_metadata_files)
train_df_metadata.columns = ['metadata_filename']
# 파일명에서 PetID를 추출. 예: '../.../a1b2c3d4-1.json' -> 'a1b2c3d4'
train_metadata_pets = train_df_metadata['metadata_filename'].apply(lambda x: x.split('/')[-1].split('-')[0])
train_df_metadata = train_df_metadata.assign(PetID=train_metadata_pets)
print(len(train_metadata_pets.unique()))

pets_with_metadatas = len(np.intersect1d(train_metadata_pets.unique(), train_df_ids['PetID'].unique()))
# 원본 데이터의 모든 반려동물 중, 메타데이터가 있는 동물의 비율을 계산하여 출력
print('fraction of pets with metadata: {:.3f}'.format(pets_with_metadatas / train_df_ids.shape[0]))


# --- 3. 감성 분석(Sentiment) 데이터 커버리지 확인 ---
# (위 데이터 처리와 과정이 거의 동일하나, PetID 추출 방식에 차이가 있음)
train_df_ids = train[['PetID']]
train_df_sentiment = pd.DataFrame(train_sentiment_files)
train_df_sentiment.columns = ['sentiment_filename']
# 파일명에서 PetID를 추출. 예: '../.../a1b2c3d4.json' -> 'a1b2c3d4'
# 여기서는 파일명에 '-'가 없으므로 '.'을 기준으로 분리
train_sentiment_pets = train_df_sentiment['filename'].apply(lambda x: x.split('/')[-1].split('.')[0])
train_df_sentiment = train_df_sentiment.assign(PetID=train_sentiment_pets)
print(len(train_sentiment_pets.unique()))

pets_with_sentiments = len(np.intersect1d(train_sentiment_pets.unique(), train_df_ids['PetID'].unique()))
# 원본 데이터의 모든 반려동물 중, 감성 분석 데이터가 있는 동물의 비율을 계산하여 출력
print('fraction of pets with sentiment: {:.3f}'.format(pets_with_sentiments / train_df_ids.shape[0]))

앞에서 보았듯이, 비정형 데이터 파일의 내부에는 어떤 내용이 있는지 바로 파악할 수 없다. <br>
따라서 "전체 동물 중 몇 퍼센트가 각 데이터를 가지고 있는가?" 즉, 데이터의 이용 가능성을 파악한다.<br>

이 과정을 통해 각 비정형 데이터가 얼마나 유용한지, 데이터가 부족한 경우 어떻게 처리할지(예: 특정 데이터는 사용하지 않거나, 없는 값을 채워 넣는 등)에 대한 분석 전략을 세울 수 있습니다.

In [ ]:
# --- 1. 테스트 세트(Test Set)의 데이터 커버리지 확인 ---
# 아래 3개의 섹션(Images, Metadata, Sentiment)은
# 이전에 Train 세트에 대해 수행했던 것과 완전히 동일한 과정입니다.
# 단지 대상이 test 데이터라는 점만 다릅니다.

# Images:
test_df_ids = test[['PetID']]
print(test_df_ids.shape)

test_df_imgs = pd.DataFrame(test_image_files)
test_df_imgs.columns = ['image_filename']
test_imgs_pets = test_df_imgs['image_filename'].apply(lambda x: x.split('/')[-1].split('-')[0])
test_df_imgs = test_df_imgs.assign(PetID=test_imgs_pets)
print(len(test_imgs_pets.unique()))

pets_with_images = len(np.intersect1d(test_imgs_pets.unique(), test_df_ids['PetID'].unique()))
# 테스트 세트의 모든 반려동물 중, 이미지가 있는 동물의 비율을 계산
print('fraction of pets with images: {:.3f}'.format(pets_with_images / test_df_ids.shape[0]))


# Metadata:
test_df_ids = test[['PetID']]
test_df_metadata = pd.DataFrame(test_metadata_files)
test_df_metadata.columns = ['metadata_filename']
test_metadata_pets = test_df_metadata['metadata_filename'].apply(lambda x: x.split('/')[-1].split('-')[0])
test_df_metadata = test_df_metadata.assign(PetID=test_metadata_pets)
print(len(test_metadata_pets.unique()))

pets_with_metadatas = len(np.intersect1d(test_metadata_pets.unique(), test_df_ids['PetID'].unique()))
# 테스트 세트의 모든 반려동물 중, 메타데이터가 있는 동물의 비율을 계산
print('fraction of pets with metadata: {:.3f}'.format(pets_with_metadatas / test_df_ids.shape[0]))


# Sentiment:
test_df_ids = test[['PetID']]
test_df_sentiment = pd.DataFrame(test_sentiment_files)
test_df_sentiment.columns = ['sentiment_filename']
test_sentiment_pets = test_df_sentiment['sentiment_filename'].apply(lambda x: x.split('/')[-1].split('.')[0])
test_df_sentiment = test_df_sentiment.assign(PetID=test_sentiment_pets)
print(len(test_sentiment_pets.unique()))

pets_with_sentiments = len(np.intersect1d(test_sentiment_pets.unique(), test_df_ids['PetID'].unique()))
# 테스트 세트의 모든 반려동물 중, 감성 분석 데이터가 있는 동물의 비율을 계산
print('fraction of pets with sentiment: {:.3f}'.format(pets_with_sentiments / test_df_ids.shape[0]))


# --- 2. 데이터 일관성 최종 확인 ---
# 이미지 파일에서 추출한 PetID 목록과 메타데이터 파일에서 추출한 PetID 목록이
# 순서까지 완전히 동일한지 확인합니다.
print('images and metadata distributions the same? {}'.format(
    np.all(test_metadata_pets == test_imgs_pets)))

위에서 수행한 작업이 train 셋에 대한 작업이었다면, 이 작업은 test 셋에 대한 작업니다.

마지막 한 줄의 코드는 데이터의 일관성을 검사하는 코드이다.
코드의 의미는
```
test_metadata_pets == test_imgs_pets: metadata 파일에서 추출한 PetID 목록과 image 파일에서 추출한 PetID 목록을 항목별로 하나하나 비교합니다. 두 목록의 길이가 같고, 같은 위치에 있는 PetID가 서로 동일하면 True, 하나라도 다르면 False를 반환합니다.

np.all(): 위 비교 결과가 모두 True일 때만 최종적으로 True를 반환합니다.
```
만약 이 결과가 True라면, 이는 **"메타데이터가 있는 모든 동물은 이미지도 가지고 있으며, 그 순서까지 완벽하게 일치한다"**는 것을 의미합니다. 이는 데이터가 매우 깨끗하고 정리가 잘 되어있다는 긍정적인 신호   


실제 Output은 TRUE

In [ ]:
# 비정형 데이터(JSON, 이미지)를 처리하고 분석 가능한 특징(feature)으로 변환하기 위한 클래스
class PetFinderParser(object):
    
    # 클래스가 생성될 때 초기 설정을 담당하는 생성자 함수
    def __init__(self, debug=False):
        
        self.debug = debug  # 디버그 모드 여부 (현재 코드에서는 사용되지 않음)
        self.sentence_sep = ' '  # 문장이나 단어를 합칠 때 사용할 구분자 (공백)
        
        # 주석: 메인 데이터프레임에 이미 'description'이 있으므로, sentiment 파일에서
        # 텍스트를 중복으로 추출할 필요가 없다는 의미.
        self.extract_sentiment_text = False # 감성 분석 파일에서 원문 텍스트를 추출할지 여부
        
        
    # --- 파일 로드 함수들 ---
    
    def open_metadata_file(self, filename):
        """
        메타데이터 JSON 파일을 열고 내용을 읽어옵니다.
        """
        with open(filename, 'r') as f: # 파일을 읽기 모드('r')로 열기
            metadata_file = json.load(f) # JSON 파일의 내용을 파이썬 딕셔너리로 변환
        return metadata_file
            
    def open_sentiment_file(self, filename):
        """
        감성 분석 JSON 파일을 열고 내용을 읽어옵니다.
        """
        with open(filename, 'r') as f: # 파일을 읽기 모드('r')로 열기
            sentiment_file = json.load(f) # JSON 파일의 내용을 파이썬 딕셔너리로 변환
        return sentiment_file
            
    def open_image_file(self, filename):
        """
        이미지 파일을 열고 numpy 배열로 변환합니다. (이 노트북에서는 직접 사용되지 않음)
        """
        image = np.asarray(Image.open(filename)) # 이미지를 숫자 배열로 변환
        return image
        
    # --- 파일 내용 파싱(Parsing) 및 특징 추출 함수들 ---
        
    def parse_sentiment_file(self, file):
        """
        감성 분석 파일(딕셔너리)을 입력받아, 감성 특징이 담긴 데이터프레임을 반환합니다.
        """
        
        # 문서 전체의 감성 점수(magnitude, score)를 추출
        file_sentiment = file['documentSentiment']
        # 텍스트에서 인식된 주요 개체(고양이, 장난감 등)의 이름을 리스트로 추출
        file_entities = [x['name'] for x in file['entities']]
        # 개체 이름들을 공백으로 구분된 하나의 문자열로 합침
        file_entities = self.sentence_sep.join(file_entities)

        # self.extract_sentiment_text가 True일 때만 실행되는 블록 (현재는 False)
        if self.extract_sentiment_text:
            # 모든 문장의 원문 텍스트를 추출하여 하나의 문자열로 합침
            file_sentences_text = [x['text']['content'] for x in file['sentences']]
            file_sentences_text = self.sentence_sep.join(file_sentences_text)
            # 모든 문장 각각의 감성 점수를 추출
            file_sentences_sentiment = [x['sentiment'] for x in file['sentences']]
            
            # 문장별 감성 점수들을 데이터프레임으로 만들고, 각 점수(magnitude, score)의 합계를 구함
            file_sentences_sentiment = pd.DataFrame.from_dict(
                file_sentences_sentiment, orient='columns').sum()
            # 계산된 점수 컬럼명에 'document_' 접두사를 추가하여 딕셔너리로 변환
            file_sentences_sentiment = file_sentences_sentiment.add_prefix('document_').to_dict()
            
            # 문서 전체 감성 점수 딕셔너리에 문장별 합산 점수를 추가
            file_sentiment.update(file_sentences_sentiment)
        
        # 최종 감성 점수 딕셔너리를 데이터프레임으로 변환
        df_sentiment = pd.DataFrame.from_dict(file_sentiment, orient='index').T
        if self.extract_sentiment_text:
            # 텍스트 추출이 True였다면, 'text' 컬럼을 추가
            df_sentiment['text'] = file_sentences_text
            
        # 개체(entities) 정보를 'entities' 컬럼으로 추가
        df_sentiment['entities'] = file_entities
        # 모든 컬럼명에 'sentiment_' 접두사를 붙여 다른 데이터와 구분
        df_sentiment = df_sentiment.add_prefix('sentiment_')
        
        return df_sentiment
    
    def parse_metadata_file(self, file):
        """
        메타데이터 파일(딕셔너리)을 입력받아, 이미지 특징이 담긴 데이터프레임을 반환합니다.
        """
        
        # 파일에 어떤 최상위 키들이 있는지 확인
        file_keys = list(file.keys())
        
        # 'labelAnnotations' 키가 있는지 확인 (이미지 라벨 정보)
        if 'labelAnnotations' in file_keys:
            # 인식된 라벨 중 상위 30%만 사용 (정보가 너무 많은 것을 방지)
            file_annots = file['labelAnnotations'][:int(len(file['labelAnnotations']) * 0.3)]
            # 상위 라벨들의 신뢰도 점수(score)의 평균을 계산
            file_top_score = np.asarray([x['score'] for x in file_annots]).mean()
            # 상위 라벨들의 설명(description)을 리스트로 저장
            file_top_desc = [x['description'] for x in file_annots]
        else:
            # 라벨 정보가 없는 경우, 점수는 NaN(결측치)으로, 설명은 빈칸으로 처리
            file_top_score = np.nan
            file_top_desc = ['']
        
        # 이미지의 지배적인 색상 정보와 구도(crop) 정보를 추출
        file_colors = file['imagePropertiesAnnotation']['dominantColors']['colors']
        file_crops = file['cropHintsAnnotation']['cropHints']

        # 각 색상의 신뢰도 점수와 픽셀 비율의 평균을 계산
        file_color_score = np.asarray([x['score'] for x in file_colors]).mean()
        file_color_pixelfrac = np.asarray([x['pixelFraction'] for x in file_colors]).mean()

        # 구도 추천 점수(confidence)의 평균을 계산
        file_crop_conf = np.asarray([x['confidence'] for x in file_crops]).mean()
        
        # 구도 정보에 'importanceFraction' 키가 있는지 확인 (없는 경우도 있음)
        if 'importanceFraction' in file_crops[0].keys():
            # 중요도(importance) 점수의 평균을 계산
            file_crop_importance = np.asarray([x['importanceFraction'] for x in file_crops]).mean()
        else:
            # 없는 경우, NaN으로 처리
            file_crop_importance = np.nan

        # 위에서 추출하고 계산한 특징들을 하나의 딕셔너리로 정리
        df_metadata = {
            'annots_score': file_top_score,
            'color_score': file_color_score,
            'color_pixelfrac': file_color_pixelfrac,
            'crop_conf': file_crop_conf,
            'crop_importance': file_crop_importance,
            'annots_top_desc': self.sentence_sep.join(file_top_desc)
        }
        
        # 정리된 딕셔너리를 데이터프레임으로 변환
        df_metadata = pd.DataFrame.from_dict(df_metadata, orient='index').T
        # 모든 컬럼명에 'metadata_' 접두사를 붙여 다른 데이터와 구분
        df_metadata = df_metadata.add_prefix('metadata_')
        
        return df_metadata
    

🔎 파싱(Parsing): 문자열이나 파일, JSON 등의 복잡한 데이터를 일정한 규칙으로 분석하고, 구조화하는 과정

1. 복잡한 딕셔너리 형태의 JSON 파일 내용을 입력받아, 그 안에서 의미 있는 정보(감성 점수, 이미지 라벨, 색상 정보 등)만 골라내고 계산(평균 등)하여, 최종적으로는 분석하기 쉬운 한 줄짜리 데이터프레임으로 만들어 반환   
파이썬 코드가 직접 감성 분석을 수행하는 것은 아니고, 그냥 json 파일을 어떻게 처리할지에 대한 규칙을 정해주는 코드이다.

In [ ]:
# --- 실제 데이터 처리 작업을 수행하는 헬퍼(Helper) 함수 ---

# PetID 하나를 입력받아 관련된 모든 추가 특징을 추출하는 함수
def extract_additional_features(pet_id, mode='train'):
    
    # Sentiment 파일 경로를 생성
    sentiment_filename = '../input/{}_sentiment/{}.json'.format(mode, pet_id)
    try:
        # 파일 열기
        sentiment_file = pet_parser.open_sentiment_file(sentiment_filename)
        # 내용 파싱하여 sentiment 데이터프레임 생성
        df_sentiment = pet_parser.parse_sentiment_file(sentiment_file)
        df_sentiment['PetID'] = pet_id
    except FileNotFoundError: # 파일을 찾지 못하면
        df_sentiment = [] # 빈 리스트로 처리

    # Metadata 파일 처리를 위한 빈 리스트 준비
    dfs_metadata = []
    # 해당 PetID로 시작하는 모든 metadata 파일을 찾음 (예: a1b2c3-1.json, a1b2c3-2.json)
    metadata_filenames = sorted(glob.glob('../input/{}_metadata/{}*.json'.format(mode, pet_id)))
    
    # 찾은 파일이 하나 이상 있다면
    if len(metadata_filenames) > 0:
        # 각 파일을 순회하며
        for f in metadata_filenames:
            # 파일 열기
            metadata_file = pet_parser.open_metadata_file(f)
            # 내용 파싱하여 metadata 데이터프레임 생성
            df_metadata = pet_parser.parse_metadata_file(metadata_file)
            df_metadata['PetID'] = pet_id
            # 처리된 결과를 dfs_metadata 리스트에 추가
            dfs_metadata.append(df_metadata)
        # 한 PetID에 여러 metadata 파일이 있을 경우, 이들을 하나의 데이터프레임으로 합침
        dfs_metadata = pd.concat(dfs_metadata, ignore_index=True, sort=False)
    
    # 최종적으로 sentiment 결과와 metadata 결과를 리스트에 담아 반환
    dfs = [df_sentiment, dfs_metadata]
    
    return dfs

2. extract_additional_features 함수: 실제 행동대장
이 함수는 위에서 만든 PetFinderParser라는 도구를 실제로 사용하는 역할을 합니다.

PetID 하나를 건네주면, 해당 ID에 맞는 sentiment와 metadata 파일들을 찾아 pet_parser에게 넘겨 처리를 맡깁니다.

try...except 구문을 사용해 파일이 없는 경우에도 오류로 멈추지 않고 유연하게 대처하며, 한 PetID에 여러 이미지(메타데이터 파일)가 있는 경우도 처리할 수 있도록 설계되었습니다.

이 함수가 바로 Parallel을 통해 수천 번씩 호출될 실제 작업 단위입니다.

In [ ]:
# 위에서 설계한 PetFinderParser 클래스의 인스턴스(실체)를 생성
# 이제 'pet_parser'라는 변수를 통해 클래스 내의 모든 함수를 사용할 수 있음
pet_parser = PetFinderParser()

3. pet_parser = PetFinderParser(): 설계도를 실체로
마지막 줄은 위에서 정의한 PetFinderParser 설계도를 바탕으로 pet_parser라는 실제 "일꾼" 객체를 만드는 과정입니다. 이제부터 pet_parser.open_metadata_file()과 같은 방식으로 클래스 안의 함수들을 호출할 수 있게 됩니다.

In [ ]:
# --- 1. 처리할 PetID 목록 준비 ---
# 디버깅 모드 플래그. False일 경우 전체 데이터를, True일 경우 일부 데이터만 사용.
debug = False
# train, test 데이터프레임에서 고유한 PetID만 추출하여 목록 생성
train_pet_ids = train.PetID.unique()
test_pet_ids = test.PetID.unique()

# 디버그 모드가 True일 경우, 실행 시간을 줄이기 위해 일부 ID만 선택
if debug:
    train_pet_ids = train_pet_ids[:1000] # train ID 1000개
    test_pet_ids = test_pet_ids[:500]   # test ID 500개


# --- 2. Train 세트 처리 ---
# Parallel을 이용한 병렬 처리 시작
# n_jobs=6: 6개의 CPU 코어를 사용
# verbose=1: 처리 진행 상황을 간단한 로그로 표시
dfs_train = Parallel(n_jobs=6, verbose=1)(
    # train_pet_ids의 모든 ID에 대해 extract_additional_features 함수를 실행
    delayed(extract_additional_features)(i, mode='train') for i in train_pet_ids
)

# --- 3. Train 세트 결과 정리 ---
# 병렬 처리 결과(dfs_train)에서 sentiment와 metadata 데이터프레임을 각각 분리
# x[0]에 있는 sentiment 결과가 데이터프레임일 경우에만 리스트에 추가
train_dfs_sentiment = [x[0] for x in dfs_train if isinstance(x[0], pd.DataFrame)]
# x[1]에 있는 metadata 결과가 데이터프레임일 경우에만 리스트에 추가
train_dfs_metadata = [x[1] for x in dfs_train if isinstance(x[1], pd.DataFrame)]

# 분리된 데이터프레임 리스트를 하나의 큰 데이터프레임으로 결합(concat)
train_dfs_sentiment = pd.concat(train_dfs_sentiment, ignore_index=True, sort=False)
train_dfs_metadata = pd.concat(train_dfs_metadata, ignore_index=True, sort=False)

# 최종적으로 생성된 데이터프레임의 형태(shape)를 출력 (행 수, 열 수)
print(train_dfs_sentiment.shape, train_dfs_metadata.shape)


# --- 4. Test 세트 처리 및 정리 ---
# (위 Train 세트 처리 과정과 완전히 동일한 로직을 Test 데이터에 적용)
dfs_test = Parallel(n_jobs=6, verbose=1)(
    delayed(extract_additional_features)(i, mode='test') for i in test_pet_ids
)

test_dfs_sentiment = [x[0] for x in dfs_test if isinstance(x[0], pd.DataFrame)]
test_dfs_metadata = [x[1] for x in dfs_test if isinstance(x[1], pd.DataFrame)]

test_dfs_sentiment = pd.concat(test_dfs_sentiment, ignore_index=True, sort=False)
test_dfs_metadata = pd.concat(test_dfs_metadata, ignore_index=True, sort=False)

print(test_dfs_sentiment.shape, test_dfs_metadata.shape)

Parallel을 이용한 병렬 처리를 통해 처리 성능 향상   

4개의 데이터셋(train_sentiment_df, train_metadata_df, test_sentiment_df, test_metadata_df) 생성, 이 4개의 df는 원본 train, test 데이터에 새로운 정보를 더해줌

In [ ]:
# 앞으로 사용할 집계(aggregation) 함수들을 리스트로 정의
aggregates = ['mean', 'sum']


# --- 1. Train 세트 - Metadata 처리 ---

# -- 1a. 텍스트 데이터(annots_top_desc) 분리 및 처리 --
# PetID를 기준으로 그룹화하고, 각 PetID의 모든 'metadata_annots_top_desc' 텍스트를 중복 없이 리스트로 묶음
train_metadata_desc = train_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
train_metadata_desc = train_metadata_desc.reset_index() # 그룹화된 인덱스를 컬럼으로 변환
# 텍스트 리스트를 공백으로 구분된 하나의 긴 문자열로 합침
train_metadata_desc[
    'metadata_annots_top_desc'] = train_metadata_desc[
    'metadata_annots_top_desc'].apply(lambda x: ' '.join(x))

# -- 1b. 수치 데이터 처리 --
prefix = 'metadata' # 컬럼명에 사용할 접두사
# 원본에서 텍스트 컬럼을 제외하여 수치 데이터만 남김
train_metadata_gr = train_dfs_metadata.drop(['metadata_annots_top_desc'], axis=1)
# PetID를 제외한 모든 컬럼을 float(실수) 타입으로 변환 (집계를 위해)
for i in train_metadata_gr.columns:
    if 'PetID' not in i:
        train_metadata_gr[i] = train_metadata_gr[i].astype(float)
# PetID를 기준으로 그룹화하고, 각 수치 컬럼에 대해 평균(mean)과 합계(sum)를 계산
train_metadata_gr = train_metadata_gr.groupby(['PetID']).agg(aggregates)
# 생성된 멀티인덱스 컬럼명을 'prefix_원본컬럼명_집계함수' 형식으로 재정의 (예: metadata_annots_score_MEAN)
train_metadata_gr.columns = pd.Index(['{}_{}_{}'.format(
            prefix, c[0], c[1].upper()) for c in train_metadata_gr.columns.tolist()])
train_metadata_gr = train_metadata_gr.reset_index() # 그룹화된 인덱스를 컬럼으로 변환


# --- 2. Train 세트 - Sentiment 처리 ---
# (위 Metadata 처리와 완전히 동일한 과정)

train_sentiment_desc = train_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
train_sentiment_desc = train_sentiment_desc.reset_index()
train_sentiment_desc[
    'sentiment_entities'] = train_sentiment_desc[
    'sentiment_entities'].apply(lambda x: ' '.join(x))

prefix = 'sentiment'
train_sentiment_gr = train_dfs_sentiment.drop(['sentiment_entities'], axis=1)
for i in train_sentiment_gr.columns:
    if 'PetID' not in i:
        train_sentiment_gr[i] = train_sentiment_gr[i].astype(float)
train_sentiment_gr = train_sentiment_gr.groupby(['PetID']).agg(aggregates)
train_sentiment_gr.columns = pd.Index(['{}_{}_{}'.format(
            prefix, c[0], c[1].upper()) for c in train_sentiment_gr.columns.tolist()])
train_sentiment_gr = train_sentiment_gr.reset_index()


# --- 3. Test 세트 처리 ---
# (위 Train 세트 처리와 완전히 동일한 과정을 Test 데이터에 적용)

# Metadata 처리
test_metadata_desc = test_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
# ... (이하 동일)
# ...

# Sentiment 처리
test_sentiment_desc = test_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
# ... (이하 동일)
# ...

- 한 반려동물이 여러 개의 파일을 가질 수 있음 -> 하나의 PetID에 여러 행의 데이터가 생기기에, 원본 train 데이터와 합치기가 곤란해질 수 있음

- 텍스트와 수치 데이터가 섞여있기 때문에 각각 다른 전략 사용
  1. 텍스트 데이터 처리 (_desc 접미사)
  여러 파일에서 나온 텍스트들을 .join() 함수를 사용해 하나의 긴 텍스트로 이어 붙임
  2. 수치 데이터 처리 (_gr 접미사)
  .agg(['mean', 'sum'])을 이용해 각 PetID별로 모든 수치 정보의 **평균(mean)과 합계(sum)**를 계산

In [ ]:
# --- 1. Train 세트 데이터 병합(Merge) ---

# 원본 train 데이터프레임의 복사본을 만들어 작업 (원본 데이터 보존)
train_proc = train.copy()
# 'PetID'를 기준으로, train_proc에 수치형 sentiment 특징(_gr)을 'left' 방식으로 병합
train_proc = train_proc.merge(
    train_sentiment_gr, how='left', on='PetID')
# 이어서 수치형 metadata 특징(_gr)을 병합
train_proc = train_proc.merge(
    train_metadata_gr, how='left', on='PetID')
# 이어서 텍스트형 metadata 특징(_desc)을 병합
train_proc = train_proc.merge(
    train_metadata_desc, how='left', on='PetID')
# 마지막으로 텍스트형 sentiment 특징(_desc)을 병합
train_proc = train_proc.merge(
    train_sentiment_desc, how='left', on='PetID')

# --- 2. Test 세트 데이터 병합(Merge) ---

# 원본 test 데이터프레임의 복사본을 만들어 작업
test_proc = test.copy()
# (위 Train 세트 병합과 동일한 과정을 Test 데이터에 적용)
test_proc = test_proc.merge(
    test_sentiment_gr, how='left', on='PetID')
test_proc = test_proc.merge(
    test_metadata_gr, how='left', on='PetID')
test_proc = test_proc.merge(
    test_metadata_desc, how='left', on='PetID')
test_proc = test_proc.merge(
    test_sentiment_desc, how='left', on='PetID')


# --- 3. 최종 결과 확인 및 검증 ---

# 모든 특징이 병합된 최종 데이터프레임의 형태(shape)를 출력
print(train_proc.shape, test_proc.shape)
# 병합 후 train 데이터의 행(row) 수가 원본과 동일한지 확인 (데이터 유실/중복 방지)
assert train_proc.shape[0] == train.shape[0]
# 병합 후 test 데이터의 행 수가 원본과 동일한지 확인
assert test_proc.shape[0] == test.shape[0]

- Data Merging 단계

- **how='left'의 중요성**
merge를 수행할 때 how='left' 옵션은 매우 중요하다. SQL의 left join과 의미는 같다.

- **assert를 이용한 검증**
assert 구문은 프로그래머의 "안전장치" 또는 "자동 점검" 기능이다. assert 뒤의 조건이 True가 아니면, AssertionError가 발생

여기서 assert train_proc.shape[0] == train.shape[0]는 "병합 후 데이터의 행 수가 원본과 정확히 일치해야만 한다"는 것을 강력하게 확인하는 과정으로 만약 병합 과정에서 무언가 잘못되어 데이터 행이 중복되거나 사라졌다면, 이 assert 구문이 즉시 문제를 알려주어 버그를 조기에 발견할 수 있게 해줌

이 단계를 거쳐 생성된 train_proc와 test_proc는 모든 특징이 결합된, 모델 학습을 위한 최종 준비를 마친 데이터셋입니다.

In [ ]:
# --- 1. Train 세트의 품종(Breed) 정보 변환 ---

# -- 1a. 첫 번째 품종(Breed1) 처리 --
# 'Breed1' 컬럼과 품종 이름이 담긴 'labels_breed'를 병합
train_breed_main = train_proc[['Breed1']].merge(
    labels_breed, how='left',         # 'Breed1'의 모든 값을 유지
    left_on='Breed1', right_on='BreedID', # 'Breed1'과 'BreedID'를 기준으로 병합
    suffixes=('', '_main_breed'))     # 중복 컬럼명에 접미사 추가

# 병합 결과에서 불필요한 ID 컬럼들(Breed1, BreedID)을 제외하고, 실제 정보가 담긴 컬럼만 남김
train_breed_main = train_breed_main.iloc[:, 2:]
# 남은 컬럼들의 이름 앞에 'main_breed_' 접두사를 붙여 첫 번째 품종 정보임을 명시
train_breed_main = train_breed_main.add_prefix('main_breed_')

# -- 1b. 두 번째 품종(Breed2) 처리 (위와 동일한 과정 반복) --
train_breed_second = train_proc[['Breed2']].merge(
    labels_breed, how='left',
    left_on='Breed2', right_on='BreedID',
    suffixes=('', '_second_breed'))

train_breed_second = train_breed_second.iloc[:, 2:]
train_breed_second = train_breed_second.add_prefix('second_breed_')


# -- 1c. 변환된 품종 정보를 원본 데이터에 최종 결합 --
# 기존 train_proc 데이터프레임 옆에(axis=1) main_breed와 second_breed 정보를 붙임
train_proc = pd.concat(
    [train_proc, train_breed_main, train_breed_second], axis=1)


# --- 2. Test 세트의 품종(Breed) 정보 변환 ---
# (위 Train 세트 처리와 완전히 동일한 과정을 Test 데이터에 적용)

test_breed_main = test_proc[['Breed1']].merge(
    labels_breed, how='left',
    left_on='Breed1', right_on='BreedID',
    suffixes=('', '_main_breed'))
# ... (이하 동일)


# --- 3. 최종 결과 확인 ---
# 품종 정보가 추가된 최종 데이터프레임의 형태(shape)를 출력
print(train_proc.shape, test_proc.shape)

- 부가 설명
이 코드 블록의 핵심 목표는 의미 없는 숫자 ID를 해석 가능한 텍스트 정보로 변환하는 특징 생성(Feature Creation) 과정입니다.

train 데이터에 있는 Breed1, Breed2 컬럼은 '307'이나 '266' 같은 숫자로만 이루어져 있습니다. 이 숫자 자체는 모델에게 아무런 의미를 주지 못합니다. 이 코드는 labels_breed.csv라는 **참조표(Lookup Table)**를 이용해 이 숫자 ID들을 'Mixed Breed', 'Domestic Short Hair' 같은 실제 품종 이름으로 바꾸어주는 역할을 합니다.

- 처리 과정 요약
정보 매칭 (merge): train_proc의 Breed1 ID를 labels_breed의 BreedID와 비교하여 일치하는 품종 이름(BreedName)과 타입(Type) 정보를 찾아 옆에 붙입니다.

정리 및 이름 부여 (iloc, add_prefix):

병합에 사용된 불필요한 ID 컬럼들을 iloc으로 제거하여 순수한 품종 정보만 남깁니다.

첫 번째 품종(Breed1)에서 온 정보인지, 두 번째 품종(Breed2)에서 온 정보인지 명확히 구분하기 위해, 각 컬럼명 앞에 main_breed_, second_breed_ 와 같은 접두사를 붙여줍니다. 이는 나중에 모델 분석 시 혼동을 막아줍니다.

최종 결합 (concat):

이렇게 깔끔하게 정리된 품종 이름 정보(train_breed_main, train_breed_second)를 원래의 train_proc 데이터프레임의 오른쪽에 열(column) 방향으로 이어 붙입니다 (axis=1).

이 과정을 통해 기존의 숫자 ID 컬럼 외에, 모델이 더 잘 이해하고 활용할 수 있는 새로운 텍스트 기반의 특징(feature) 컬럼들이 데이터셋에 추가됩니다.

In [ ]:
X = pd.concat([train_proc, test_proc], ignore_index=True, sort=False)
print('NaN structure:\n{}'.format(np.sum(pd.isnull(X))))

- 본격적인 모델링에 앞서 데이터를 최종적으로 통합하고 마지막으로 점검

- 왜 합쳤는가?   

  - 머신러닝 모델을 만들기 전, 데이터를 전처리(예: 텍스트 인코딩, 숫자 스케일링 등)해야 합니다. 이때 train 데이터와 test 데이터는 반드시 동일한 기준과 방법으로 처리되어야 합니다.

  - 만약 두 데이터를 따로따로 전처리하면, 기준이 미세하게 달라져 모델 성능에 문제가 생길 수 있습니다.

  - 그래서 가장 효율적이고 안전한 방법은 다음과 같습니다.
    1. 일단 합친다 (pd.concat): train과 test를 하나의 큰 데이터(X)로 합칩니다.
    2. 한 번에 처리한다: 합쳐진 데이터 X에 모든 전처리 과정을 한꺼번에 적용합니다.
    3. 다시 나눈다: 전처리가 끝나면, AdoptionSpeed 열에 값이 있는지 없는지를 기준으로 다시 X_train과 X_test로 분리합니다.

In [ ]:
# --- 1. 데이터 타입별로 컬럼 분류 ---
# 통합 데이터프레임 X의 각 컬럼별 데이터 타입(dtype)을 확인
column_types = X.dtypes

# 데이터 타입이 정수(int)인 컬럼들만 선택
int_cols = column_types[column_types == 'int']
# 데이터 타입이 실수(float)인 컬럼들만 선택
float_cols = column_types[column_types == 'float']
# 데이터 타입이 객체(object)인 컬럼들만 선택 (주로 문자열 데이터)
cat_cols = column_types[column_types == 'object']

# --- 2. 분류 결과 출력 ---
# 분류된 정수형 컬럼들의 목록을 출력
print('\tinteger columns:\n{}'.format(int_cols))
# 분류된 실수형 컬럼들의 목록을 출력
print('\n\tfloat columns:\n{}'.format(float_cols))
# 분류된 범주형(object) 컬럼들의 목록을 출력. 이 컬럼들은 인코딩이 필요함.
print('\n\tto encode categorical columns:\n{}'.format(cat_cols))

- 각 컬럼들의 데이터 타입을 구분해두면 후처리가 쉬워진답니다~ (수치형 컬럼들에는 스케일링을, 범주형 컬럼들에는 인코딩을 적용)

In [ ]:
# Copy original X DF for easier experimentation,
# all feature engineering will be performed on this one:
X_temp = X.copy()


# Select subsets of columns:
text_columns = ['Description', 'metadata_annots_top_desc', 'sentiment_entities']
categorical_columns = ['main_breed_BreedName', 'second_breed_BreedName']

# Names are all unique, so they can be dropped by default
# Same goes for PetID, it shouldn't be used as a feature
to_drop_columns = ['PetID', 'Name', 'RescuerID']
# RescuerID will also be dropped, as a feature based on this column will be extracted independently

- text_columns: 자연어 처리(NLP) 기술(예: TF-IDF)을 적용해야 하는 컬럼들입니다.

- categorical_columns: 품종 이름처럼 정해진 몇 개의 카테고리로 이루어진 컬럼들입니다. 이들은 주로 원-핫 인코딩(One-Hot Encoding)으로 처리됩니다.

- to_drop_columns: 모델의 예측 성능에 도움이 되지 않거나, 오히려 방해가 될 수 있는 컬럼들을 미리 지정합니다. PetID나 Name처럼 모든 행마다 값이 다른 고유 식별자는 모델이 암기하게 만들어 과적합(overfitting)을 유발할 수 있으므로 제거하는 것이 일반적입니다.

In [ ]:
# Count RescuerID occurrences:
rescuer_count = X.groupby(['RescuerID'])['PetID'].count().reset_index()
rescuer_count.columns = ['RescuerID', 'RescuerID_COUNT']

# Merge as another feature onto main DF:
X_temp = X_temp.merge(rescuer_count, how='left', on='RescuerID')

- RescuerID_COUNT 라는 파생 변수 생성

In [ ]:
# Factorize categorical columns:
for i in categorical_columns:
    X_temp.loc[:, i] = pd.factorize(X_temp.loc[:, i])[0]

- 라벨인코딩

In [ ]:
# Subset text features:
X_text = X_temp[text_columns]

for i in X_text.columns:
    X_text.loc[:, i] = X_text.loc[:, i].fillna('<MISSING>')

이 코드 블록은 **자연어 처리(NLP)**를 위한 텍스트 데이터를 따로 준비하고, 처리 과정에서 오류를 방지하기 위해 결측치를 관리하는 단계입니다.

텍스트 데이터 분리: 전체 데이터(X_temp)에서 텍스트 분석이 필요한 컬럼들만 골라 X_text라는 별도의 데이터프레임으로 만드는 것은 효율적인 작업 방식입니다. 이렇게 하면 숫자나 다른 종류의 데이터에 영향을 주지 않고 텍스트 관련 전처리(예: TF-IDF)를 깔끔하게 적용할 수 있습니다.

결측치 처리의 중요성: 텍스트 데이터를 벡터화하는 라이브러리(예: TfidfVectorizer)는 NaN과 같은 비어있는 값을 만나면 오류를 일으킵니다. 따라서 모든 값을 문자열 형태로 만들어주어야 합니다. .fillna('<MISSING>')은 이러한 NaN 값들을 '<MISSING>'이라는 일관된 문자열로 대체하여 후속 작업이 원활하게 진행되도록 보장합니다.

또한, 단순히 빈칸('')으로 채우는 대신 '<MISSING>'이라는 의미 있는 문자열로 채우면, "설명이 없는 경우" 자체가 하나의 중요한 특징(feature)으로 작용하여 모델이 학습하는 데 도움이 될 수도 있습니다.

In [ ]:
# --- 1. 텍스트 특징 추출을 위한 라이브러리 임포트 ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF # 차원 축소 기법

# --- 2. 파라미터 및 변수 초기화 ---
n_components = 5      # 각 텍스트 컬럼에서 추출할 주요 토픽(topic) 또는 특징(feature)의 개수
text_features = []    # 생성된 텍스트 특징들을 임시로 저장할 리스트

# --- 3. 각 텍스트 컬럼별 특징 생성 ---
# 이전에 준비한 텍스트 컬럼들('Description' 등)에 대해 반복
for i in X_text.columns:
    
    # -- 3a. 알고리즘 초기화 --
    print('generating features from: {}'.format(i))
    # Truncated SVD 모델 초기화 (n_components개의 특징 추출)
    svd_ = TruncatedSVD(n_components=n_components, random_state=1337)
    # NMF 모델 초기화 (n_components개의 특징 추출)
    nmf_ = NMF(n_components=n_components, random_state=1337)
    
    # -- 3b. TF-IDF 벡터화 --
    # TfidfVectorizer를 이용해 텍스트 데이터를 수치 벡터(TF-IDF 행렬)로 변환
    tfidf_col = TfidfVectorizer().fit_transform(X_text.loc[:, i].values)
    
    # -- 3c. SVD 적용 및 데이터프레임 변환 --
    # TF-IDF 행렬에 SVD를 적용하여 n_components개의 주요 특징으로 압축
    svd_col = svd_.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col) # 결과를 데이터프레임으로 변환
    svd_col = svd_col.add_prefix('SVD_{}_'.format(i)) # 컬럼명에 접두사를 붙여 구분
    
    # -- 3d. NMF 적용 및 데이터프레임 변환 --
    # TF-IDF 행렬에 NMF를 적용하여 n_components개의 주요 특징으로 압축
    nmf_col = nmf_.fit_transform(tfidf_col)
    nmf_col = pd.DataFrame(nmf_col) # 결과를 데이터프레임으로 변환
    nmf_col = nmf_col.add_prefix('NMF_{}_'.format(i)) # 컬럼명에 접두사를 붙여 구분
    
    # -- 3e. 결과 저장 --
    # 생성된 SVD와 NMF 특징을 text_features 리스트에 추가
    text_features.append(svd_col)
    text_features.append(nmf_col)

    
# --- 4. 모든 텍스트 특징 결합 ---
# text_features 리스트에 저장된 모든 데이터프레임 조각들을 옆으로 이어 붙임 (axis=1)
text_features = pd.concat(text_features, axis=1)

# --- 5. 최종 데이터프레임에 병합 및 원본 텍스트 제거 ---
# 생성된 텍스트 특징들을 메인 데이터프레임(X_temp)에 결합
X_temp = pd.concat([X_temp, text_features], axis=1)

# 원본 텍스트 컬럼들은 이제 불필요하므로 제거
for i in X_text.columns:
    X_temp = X_temp.drop(i, axis=1)

이 코드 블록은 **자연어 처리(NLP)**의 핵심적인 부분으로, 컴퓨터가 이해할 수 없는 텍스트 데이터를 의미 있는 숫자형 특징(feature)으로 변환하는 과정입니다. 이 과정은 크게 두 단계로 나뉩니다.

1. 텍스트의 수치화: TfidfVectorizer
컴퓨터는 '고양이'라는 단어 자체를 이해하지 못합니다. **TF-IDF(Term Frequency-Inverse Document Frequency)**는 각 텍스트(문서)에서 단어의 중요도를 계산하여, 텍스트 데이터를 거대한 숫자 행렬로 변환하는 기법입니다.

TF (Term Frequency): 한 문서 안에서 특정 단어가 얼마나 자주 등장하는지.

IDF (Inverse Document Frequency): 특정 단어가 전체 문서들 중에서 얼마나 희귀하게 등장하는지. (모든 문서에 다 나오는 단어는 중요도가 낮음)

이 과정을 거치면 각 텍스트는 수만 개의 단어(컬럼)로 이루어진 매우 큰 숫자 벡터가 됩니다.

2. 정보 압축 및 토픽 추출: TruncatedSVD & NMF
TF-IDF로 변환된 데이터는 너무 크고(수만 개의 컬럼) 정보가 흩어져 있어 모델이 학습하기 어렵습니다. 차원 축소(Dimensionality Reduction) 기법은 이 방대한 정보를 몇 개의 핵심적인 '토픽(Topic)' 또는 **'잠재 의미(Latent Semantic)'**로 압축하는 역할을 합니다.

TruncatedSVD (특이값 분해): 텍스트에 숨어있는 주요 패턴과 의미를 찾아내어, n_components개(여기서는 5개)의 가장 중요한 특징으로 정보를 요약합니다.

NMF (음수 미포함 행렬 분해): SVD와 유사하게 토픽을 추출하지만, 모든 값이 양수여야 한다는 제약 조건이 있어 좀 더 해석이 쉬운 경향이 있습니다.

결과적으로, 하나의 긴 반려동물 설명문은 "활발함에 대한 점수", "친근함에 대한 점수" 등과 같이 5개의 주요 토픽 점수로 압축됩니다. 이 코드에서는 SVD와 NMF 두 가지 기법을 모두 사용하여 다양한 관점의 특징을 추출하고 있습니다.

이 과정을 거쳐 원본 텍스트 컬럼들은 모델이 학습할 수 있는 10개(= 5개(SVD) + 5개(NMF))의 새로운 숫자형 특징 컬럼으로 대체됩니다.

- "의미를 알 수 없는 긴 텍스트" → TF-IDF → "수만 개의 단어 점수 벡터" → SVD/NMF → "모델이 학습할 수 있는 5개의 핵심 토픽 점수"

In [ ]:
# Remove unnecessary columns:
X_temp = X_temp.drop(to_drop_columns, axis=1)

# Check final df shape:
print('X shape: {}'.format(X_temp.shape))

- 불필요한 컬럼 제거

In [ ]:
# --- 1. 데이터를 다시 Train과 Test로 분리 ---

# 'AdoptionSpeed' 컬럼에 유한한 값(숫자)이 있는 행들을 X_train으로 선택 (원본 train 데이터)
X_train = X_temp.loc[np.isfinite(X_temp.AdoptionSpeed), :]
# 'AdoptionSpeed' 컬럼에 유한한 값이 없는 행(NaN)들을 X_test로 선택 (원본 test 데이터)
X_test = X_temp.loc[~np.isfinite(X_temp.AdoptionSpeed), :]

# --- 2. Test 세트에서 정답 컬럼 제거 ---

# X_test 데이터프레임에는 정답이 없어야 하므로, 'AdoptionSpeed' 컬럼을 제거
X_test = X_test.drop(['AdoptionSpeed'], axis=1)


# --- 3. 분리 결과 확인 및 검증 ---

# 최종적으로 분리된 X_train과 X_test의 형태(shape)를 출력
print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))

# 분리된 X_train의 행 수가 원본 train의 행 수와 동일한지 확인
assert X_train.shape[0] == train.shape[0]
# 분리된 X_test의 행 수가 원본 test의 행 수와 동일한지 확인
assert X_test.shape[0] == test.shape[0]


# --- 4. 두 데이터프레임의 컬럼 일치 여부 확인 ---

# X_train의 컬럼 목록에서 정답('AdoptionSpeed') 컬럼을 제외
train_cols = X_train.columns.tolist()
train_cols.remove('AdoptionSpeed')

# X_test의 컬럼 목록을 가져옴
test_cols = X_test.columns.tolist()

# 두 컬럼 목록이 순서까지 완벽하게 동일한지 확인
assert np.all(train_cols == test_cols)

- 모델 학습을 위해 다시 train과 test 세트로 분리

In [ ]:
np.sum(pd.isnull(X_train))

- train 결측치 개수 최종 확인

In [ ]:
np.sum(pd.isnull(X_test))

- test 결측치 개수 최종 확인

In [ ]:
# --- 1. 라이브러리 임포트 ---
import scipy as sp  # 과학 계산, 특히 최적화(optimize)를 위해 사용

from collections import Counter # 데이터의 요소 개수를 세는 기능
from functools import partial   # 함수의 인자 중 일부를 고정하여 새로운 함수를 만드는 기능
from math import sqrt           # 제곱근 계산

# Scikit-learn에서 평가 지표(metrics) 관련 함수들을 임포트
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix


# --- 2. 평가 지표(Quadratic Weighted Kappa) 계산 함수 ---
# 원본 출처: https://github.com/benhamner/Metrics

def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    두 평가자(실제값, 예측값) 간의 혼동 행렬(Confusion Matrix)을 생성합니다.
    """
    assert(len(rater_a) == len(rater_b)) # 두 평가값의 개수가 동일한지 확인
    # 최소/최대 평점(rating)이 주어지지 않으면 데이터에서 자동으로 찾음
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    # 평점의 종류 개수 계산 (0, 1, 2, 3, 4 -> 5개)
    num_ratings = int(max_rating - min_rating + 1)
    # N x N 크기의 0으로 채워진 행렬 생성
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    # 실제값(a)과 예측값(b)을 하나씩 비교하며 해당 위치의 카운트를 1씩 증가
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    주어진 평점(rating) 데이터의 분포(히스토그램)를 계산합니다.
    """
    # ... (confusion_matrix와 유사한 로직으로 각 평점이 몇 번 나왔는지 셈)
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    이 대회의 공식 평가 지표인 Quadratic Weighted Kappa 점수를 계산합니다.
    실제값(y)과 예측값(y_pred)이 얼마나 일치하는지를 측정합니다. (-1 ~ 1 사이의 값)
    단순히 맞고 틀림을 넘어, '얼마나 심하게' 틀렸는지에 가중치를 두어 점수를 매깁니다.
    """
    # ... (내부적으로 위의 confusion_matrix, histogram 함수를 사용하여 복잡한 가중치 계산 수행)
    # ...
    return (1.0 - numerator / denominator)

# --- 3. 최적의 분류 경계값을 찾는 클래스 ---
class OptimizedRounder(object):
    """
    회귀 모델이 예측한 연속적인 값(예: 2.3, 1.8)을
    어떤 경계값(threshold)으로 잘라야 가장 높은 Kappa 점수를 얻을 수 있는지
    그 최적의 경계값을 찾아주는 클래스입니다.
    """
    def __init__(self):
        self.coef_ = 0 # 최적의 경계값(계수)을 저장할 변수

    # Kappa 점수를 '손실 함수'로 변환하는 내부 함수 (최적화 라이브러리는 손실을 최소화하므로)
    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        # 주어진 경계값(coef)을 기준으로 예측값(X_p)을 0, 1, 2, 3, 4의 범주로 변환
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            # ... (이하 생략) ...
            else:
                X_p[i] = 4

        # 변환된 예측값으로 Kappa 점수를 계산
        ll = quadratic_weighted_kappa(y, X_p)
        # Kappa 점수는 높을수록 좋으므로, 앞에 -를 붙여 '손실'로 변환 (최소화 문제로 만들기 위함)
        return -ll

    # 최적의 경계값을 찾는 메인 함수
    def fit(self, X, y):
        # 최적화할 손실 함수를 준비 (X와 y 값을 고정)
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        # 초기 경계값 설정
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        # scipy의 최적화 함수(minimize)를 사용해 손실(-kappa)을 최소화하는 경계값을 찾음
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    # 찾아낸 최적의 경계값으로 새로운 데이터를 예측하는 함수
    def predict(self, X, coef):
        X_p = np.copy(X)
        # ... (_kappa_loss와 동일한 로직으로 예측) ...
        return X_p

    # 최적화 결과로 찾아낸 경계값(계수)을 반환하는 함수
    def coefficients(self):
        return self.coef_['x']

# --- 4. RMSE 계산 함수 ---
def rmse(actual, predicted):
    """
    평균 제곱근 오차(Root Mean Squared Error)를 계산하는 간단한 함수.
    """
    return sqrt(mean_squared_error(actual, predicted))

- 대회의 공식 평가 지표인 Quadratic Weighted Kappa 지표의 점수가 잘 나올수 있도록 계산 로직을 미리 설정

-  OptimizedRounder: 회귀 모델을 위한 '필살기'

많은 경우, 입양 속도(0, 1, 2, 3, 4)를 직접 예측하는 분류(Classification) 문제보다, 연속적인 값(예: 2.7, 3.1)을 예측하는 회귀(Regression) 문제로 접근하는 것이 성능이 더 좋을 때가 많습니다.

하지만 회귀 모델의 예측값(2.7, 3.1)은 최종적으로 0, 1, 2, 3, 4 중 하나로 변환해야 합니다. 이때 단순히 반올림(예: 2.7 -> 3)하는 것보다 더 좋은 방법이 있습니다.

OptimizedRounder 클래스는 바로 이 **최적의 변환 경계(Threshold)**를 찾아주는 역할을 합니다.

작동 원리: scipy.optimize.minimize라는 최적화 도구를 사용하여, 어떤 경계값(예: [1.7, 2.5, 3.2, 3.8])으로 예측값을 잘라야 Kappa 점수가 가장 높아지는지를 자동으로 탐색합니다.

효과: 단순히 반올림하는 것보다 훨씬 더 높은 최종 점수를 얻게 해주는 매우 중요한 후처리(Post-processing) 기법입니다.

- 코드 블록 전체는 모델링의 핵심 로직이라기보다는, 모델의 성능을 정확히 측정하고 그 성능을 극한까지 끌어올리기 위한 고급 유틸리티 및 전략이라고 할 수 있습니다.

In [ ]:
import lightgbm as lgb

params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17}

# Additional parameters:
early_stop = 500
verbose_eval = 100
num_rounds = 10000
n_splits = 5
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=n_splits, random_state=1337)


oof_train = np.zeros((X_train.shape[0]))
oof_test = np.zeros((X_test.shape[0], n_splits))


i = 0
for train_index, valid_index in kfold.split(X_train, X_train['AdoptionSpeed'].values):
    
    X_tr = X_train.iloc[train_index, :]
    X_val = X_train.iloc[valid_index, :]
    
    y_tr = X_tr['AdoptionSpeed'].values
    X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)
    
    y_val = X_val['AdoptionSpeed'].values
    X_val = X_val.drop(['AdoptionSpeed'], axis=1)
    
    print('\ny_tr distribution: {}'.format(Counter(y_tr)))
    
    d_train = lgb.Dataset(X_tr, label=y_tr)
    d_valid = lgb.Dataset(X_val, label=y_val)
    watchlist = [d_train, d_valid]
    
    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)
    
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    oof_train[valid_index] = val_pred
    oof_test[:, i] = test_pred
    
    i += 1

oof_train, oof_test의 역할
oof_train (Out-of-Fold train): 각 데이터가 검증용으로 사용되었을 때의 예측값을 모아놓은 것입니다. 이는 모델이 한 번도 보지 못한 데이터에 대한 예측이므로, 이 oof_train과 실제 정답을 비교하면 모델의 일반화 성능을 가장 객관적으로 측정할 수 있습니다.

oof_test: test 데이터에 대한 5개 모델의 예측값을 모두 저장합니다. 최종 제출 파일을 만들 때 이 값들의 평균을 사용하게 됩니다.

In [ ]:
plt.hist(oof_train)

1. 예측값의 중심 경향: 예측값들이 특정 값(예: 2.5) 주변에 많이 몰려 있는지, 아니면 넓게 퍼져 있는지를 시각적으로 파악할 수 있습니다.

2. 분포의 형태: 분포가 정규분포와 비슷한지, 아니면 한쪽으로 치우쳐져 있는지 등을 확인할 수 있습니다.

3. 경계값 설정의 단서: 이 히스토그램을 보고, 다음 단계인 OptimizedRounder를 사용할 때 최적의 분류 경계값이 대략 어디쯤 위치할지 직관적인 힌트를 얻을 수 있습니다.

In [ ]:
# --- 1. 최적의 분류 경계값(Threshold) 찾기 ---
# 이전에 정의한 OptimizedRounder 클래스의 인스턴스를 생성
optR = OptimizedRounder()
# OOF 예측값(oof_train)과 실제 정답(AdoptionSpeed)을 사용해 Kappa 점수를 최대화하는 경계값을 찾음
optR.fit(oof_train, X_train['AdoptionSpeed'].values)
# 위에서 찾은 최적의 경계값을 'coefficients' 변수에 저장
coefficients = optR.coefficients()

# --- 2. 최적 경계값을 적용하여 최종 예측 생성 ---
# 찾은 경계값(coefficients)을 oof_train 예측값에 적용하여 최종 분류(0~4) 결과를 생성
pred_test_y_k = optR.predict(oof_train, coefficients)

# --- 3. 결과 확인 및 최종 점수 계산 ---
# 실제 정답값들의 분포를 출력
print("\nValid Counts = ", Counter(X_train['AdoptionSpeed'].values))
# 최적화된 예측값들의 분포를 출력
print("Predicted Counts = ", Counter(pred_test_y_k))
# 찾아낸 최적의 경계값들을 출력
print("Coefficients = ", coefficients)
# 최종 예측 결과와 실제 정답 사이의 Quadratic Weighted Kappa 점수를 계산
qwk = quadratic_weighted_kappa(X_train['AdoptionSpeed'].values, pred_test_y_k)
# 최종 QWK 점수를 출력
print("QWK = ", qwk)

단순 반올림 대신 데이터에 최적화된 기준을 찾아 회귀 예측을 분류로 변환함으로써, 모델의 잠재 성능을 최대한으로 끌어내는 매우 중요한 후처리 과정

In [ ]:
# Manually adjusted coefficients:

coefficients_ = coefficients.copy()

coefficients_[0] = 1.645 # 첫 번째 경계선(0과 1을 나누는 선)의 위치를 1.645로 옮긴다.
coefficients_[1] = 2.115 # 두 번째 경계선(1과 2를 나누는 선)의 위치를 2.115로 옮긴다.
coefficients_[3] = 2.84 # 세 번째 경계선(3과 4를 나누는 선)의 위치를 2.84로 옮긴다.

train_predictions = optR.predict(oof_train, coefficients_).astype(int)
print('train pred distribution: {}'.format(Counter(train_predictions)))

test_predictions = optR.predict(oof_test.mean(axis=1), coefficients_)
print('test pred distribution: {}'.format(Counter(test_predictions)))

```
<-- 등급 0 --|-- 등급 1 --|-- 등급 2 --|-- 등급 3 --|-- 등급 4 -->
             ↑           ↑            ↑            ↑
        경계선 1      경계선 2     경계선 3      경계선 4
   (coefficients[0]) (coefficients[1]) (coefficients[2]) (coefficients[3])
```

- 임의로 최적값 수정

- 국소 최적해 (Local Minimum): 최적화 알고리즘이 완벽한 전역 최적해(Global Minimum)가 아닌, 그럴듯한 국소 최적해에 머물렀을 수 있습니다.

- 데이터의 미묘한 특성: OOF 예측값의 분포나 실제 정답의 분포를 시각화해 본 분석가가 "경계값을 약간만 옮기면 더 안정적인 예측이 가능하겠다"고 판단할 수 있습니다. 예를 들어, 특정 경계값 근처에 예측값들이 너무 빽빽하게 모여 있다면, 약간의 조정으로 분류 결과가 더 안정될 수 있습니다.

국소 최적해는 주로 다음과 같은 경우에 발생합니다.

1. 문제 자체가 복잡하고 울퉁불퉁할 때 (Non-convex Problems)
가장 근본적인 원인입니다. 해결하려는 문제의 정답을 찾는 과정이 평탄한 언덕이 아니라, 여러 개의 봉우리와 계곡이 있는 험준한 산맥을 탐험하는 것과 같은 경우입니다.

비유: 가장 높은 산봉우리(전체 최적해)를 찾는 것이 목표인데, 안개가 자욱해서 주변만 보입니다. 일단 가장 높은 곳으로 계속 올라가다 보니 어떤 봉우리(국소 최적해)에 도착했습니다. 하지만 안개 때문에 저 멀리 더 높은 진짜 정상(전체 최적해)이 있다는 것을 알지 못하고 탐험을 멈추게 됩니다.

실제 예시: 딥러닝 모델의 학습 과정, 복잡한 시스템의 최적화 문제 등 수많은 변수들이 서로 얽혀있는 문제들이 여기에 해당합니다.

2. 알고리즘이 '탐욕적(Greedy)'일 때
알고리즘이 전체적인 큰 그림을 보지 않고, 매 순간 눈앞의 이익이 가장 큰 방향으로만 움직이는 방식일 때 국소 최적해에 빠지기 쉽습니다.

비유: 동네에서 가장 높은 곳으로 가려고 합니다. 다른 길은 보지 않고, 무조건 현재 위치에서 가장 경사가 가파른 오르막길로만 계속 올라갑니다. 그러다 결국 동네 뒷산 정상에는 도착했지만, 도시 전체에서 가장 높은 남산타워로 가는 길은 놓치게 됩니다.

실제 예시: 포트폴리오 예시처럼 "일단 베타 낮은 것만 고르고, 그 안에서 수익률 제일 높은 것"과 같이 단계별로 최선을 선택하는 방식, 일부 클러스터링 알고리즘 등이 여기에 해당합니다.

3. 시작 지점이 좋지 않을 때
최적화 알고리즘은 보통 임의의 지점에서 탐색을 시작합니다. 이때 어디서 시작했는지에 따라 최종적으로 도달하는 봉우리가 달라질 수 있습니다.

비유: 헬리콥터가 두 명의 탐험가를 산맥의 서로 다른 지점에 내려주었습니다. 두 사람 모두 각자의 위치에서 가장 높은 봉우리를 향해 올라갔지만, 시작 위치가 달랐기 때문에 결국 서로 다른 봉우리에 도달하게 됩니다.

실제 예시: K-평균 클러스터링(K-Means Clustering)은 처음에 중심점을 어디에 찍느냐에 따라 최종 클러스터 결과가 크게 달라집니다. 이를 방지하기 위해 여러 번 다른 시작점에서 실행해보는 방법을 사용합니다.

4. 규칙(모델)이 너무 단순할 때 (사용자께서 지적하신 경우)
문제의 복잡성을 충분히 담아내지 못하는 지나치게 단순한 모델이나 규칙을 사용하면, 다양한 가능성을 탐색하지 못하고 뻔한 결론에 도달하게 됩니다.

비유: "가장 돈을 많이 버는 방법은?"이라는 질문에 "가장 월급을 많이 주는 회사 한 군데에 취직하는 것"이라고 답하는 것과 같습니다. 사업, 투자, 부업 등 다양한 조합을 통한 최적의 부 창출 방법을 고려하지 못한 것입니다.

실제 예시: 사용자께서 경험하신 포트폴리오 '몰빵' 추천이 바로 이 경우에 해당합니다. 여러 자산을 조합했을 때의 위험 분산 효과라는 중요한 규칙을 모델이 고려하지 않은 것입니다.

In [ ]:
# Distribution inspection of original target and predicted train and test:

print("True Distribution:")
print(pd.value_counts(X_train['AdoptionSpeed'], normalize=True).sort_index())
print("\nTrain Predicted Distribution:")
print(pd.value_counts(train_predictions, normalize=True).sort_index())
print("\nTest Predicted Distribution:")
print(pd.value_counts(test_predictions, normalize=True).sort_index())

- 모델이 예측한 결과와 실제 데이터의 분포 조정

In [ ]:
# Generate submission:

submission = pd.DataFrame({'PetID': test['PetID'].values, 'AdoptionSpeed': test_predictions.astype(np.int32)})
submission.head()
submission.to_csv('submission.csv', index=False)